In [2]:
import pickle
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

In [3]:
def load_pickle(file_path):
    try:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
def discretize_action_to_control_mode_E2E(action):
    # Your action discretization logic here
    action_norm = (action + 1) / 2
    if 1 / 6 > action_norm >= 0:
        control_mode = 0
        friction_state = 1  # left finger high friction
    elif 2 / 6 > action_norm >= 1 / 6:
        control_mode = 1
        friction_state = 1
    elif 3 / 6 > action_norm >= 2 / 6:
        control_mode = 2
        friction_state = -1
    elif 4 / 6 > action_norm >= 3 / 6:
        control_mode = 3
        friction_state = -1
    elif 5 / 6 > action_norm >= 4 / 6:
        control_mode = 4
        friction_state = 0
    else:
        assert 1 >= action_norm >= 5 / 6
        control_mode = 5
        friction_state = 0
    return friction_state, control_mode

In [7]:
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Diffusion1/hopper-medium-v2.pkl'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-1686demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-1817demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-1135demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-2mmPolicy_5mmThreshold'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-2mmPolicy_2mmThreshold'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/VFF-random'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/VFF-random_9884demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/VFF-random_withoutRotation_9983demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/VFF-test'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/VFF-bigSteps-1879demos-2mmPolicy-5mmThreshold'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-5000demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-10000demos'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Diffusion1/get_stacked_dataset/VFF-bigSteps_stacked_4_2_2.pkl'
# pickle_file_path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/VFF-test'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-2000demos-randomise'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF-bigSteps-2000demos-randomise_1_repeat'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10000demos_slide'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-10000demos'
# pickle_file_path =  '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-10000demos'
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10000demos_slide'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10000demos_slide_endIndicator'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF_bigSteps_20kDataset_seed_1'
# pickle_file_path = '/Users/qiyangyan/Desktop/TrainingFiles/Real4/Real4/demonstration/VFF-test'
# pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/bigSteps_10k_demos_random_slide'
data = load_pickle(pickle_file_path)

In [8]:
test = data['terminals'].copy()
print(np.expand_dims(test, axis=1))
print(np.shape(data['observations']))

In [15]:
for i, item in enumerate(data['actions']):
    action_mode = discretize_action_to_control_mode_E2E(item[1])[1]
    data['actions'][i][1] = action_mode

print(data['actions'][:20])

In [713]:
# print(data['desired_goals'][:10])
# print(data['sampled_desired_goals'][:10])
print(data['actions'][0])
print((data['actions'][0][0] + 1) / 2 * 1.8807)
print("Start: ", data['observations'][0][0], data['observations'][0][2], data['observations'][1][2])
print(abs(data['observations'][0][0] - data['observations'][1][0]))
print(abs(data['observations'][0][2] - data['observations'][1][2]))

In [714]:
# with open(f'/Users/qiyangyan/Desktop/Training Files/Diffusion1/big_steps_stack_dataset_4_2_2.pkl', 'rb') as f:
#   dataset_test = pickle.load(f)
# 
# print(dataset_test.__getitem__(0)['action'])
# print(dataset_test.__getitem__(0)['obs'])
# 
# print(data['actions'].shape)
# print(dataset_test.__len__())
# # print(dataset_test.indices)
# 
# for idx in range(dataset_test.__len__()):
    

In [715]:
if data is not None:
    print("Type of the loaded data:", type(data))
    if isinstance(data, dict):
        print("Keys in the dictionary:", data.keys())
    elif isinstance(data, list):
        print("Number of elements in the list:", len(data))
    elif isinstance(data, (set, tuple)):
        print("Number of elements:", len(data))
    else:
        print("Data:", data)

In [716]:
print(np.mean(data['desired_goals'], axis=0))
print(np.shape(data['desired_goals']))

In [717]:
def normalize(data, mean, std):
    # Ensure standard deviation is not zero to avoid division by zero error
    std = np.where(std == 0, 1, std)
    return (data - mean) / std

def normalize_max_min(data, min, max):
    # Ensure standard deviation is not zero to avoid division by zero error
    mask = (max == min)
    # data_norm = np.where(mask, 0.5, (data - min) / (max - min))
    data_norm = np.divide(data - min, max - min, where=~mask, out=np.full(data.shape, np.nan))
    data_norm = np.where(mask, 0.5, data_norm)
    return data_norm * 2 - 1

In [718]:
action_max = np.max(data['desired_goals'], axis=0)
action_min = np.min(data['desired_goals'], axis=0)
mask = (action_max == action_min)
print(mask)

action_norm = normalize_max_min(data['desired_goals'], min=action_min, max=action_max)
# print(action_max)
# print(action_min)
# print(np.shape(action_norm))
# print(np.max(action_norm, axis=0))
# print(np.min(action_norm, axis=0))
print(action_norm[0])

In [719]:
action_mean = np.mean(data['actions'], axis=0)
action_std = np.std(data['actions'], axis=0)
action_norm = normalize(data['actions'], action_mean, action_std)
print(np.mean(data['actions'][:, 1]))
print(np.max(data['actions'], axis=0))
print(np.min(data['actions'], axis=0))
print(action_mean, action_std)
print(np.shape(action_norm))
print(np.max(action_norm, axis=0))
print(np.min(action_norm, axis=0))
# print(np.min(action_norm))

Normalize

In [720]:
def get_data_stats(data_):
    data_ = data_.reshape(-1,data_.shape[-1])
    stats = {
        'min': np.min(data_, axis=0),
        'max': np.max(data_, axis=0)
    }
    return stats

def normalize_data(data_, stats):
    # nomalize to [0,1]
    # epsilon = 1e-10
    ndata = (data_ - stats['min']) / (stats['max'] - stats['min'])
    # normalize to [-1, 1]
    ndata = ndata * 2 - 1
    return ndata

stats = {}
normalized_train_data = {key: [] for key in data.keys()}
for key, sub_data in data.items():
    if key == 'terminals' or key == 'actions' or key == 'rewards' or key == 'next_observations':
        continue
    stats[key] = get_data_stats(sub_data)
    normalized_train_data[key] = normalize_data(sub_data, stats[key])
    print(key, stats[key])
    
for key in normalized_train_data.keys():
    if key == 'terminals' or key == 'actions' or key == 'rewards' or key == 'next_observations':
        continue
    print(np.shape(data[key]))
    print(np.shape(normalized_train_data[key]))
    print(key)
    print('min', np.min(normalized_train_data[key], axis=0))
    print('max', np.max(normalized_train_data[key], axis=0))
    
print(normalized_train_data['desired_goals'][-1][2])


In [721]:
for key in data.keys():
    print(np.shape(data[key]), key)
    # print(data[key][0])
    
print(data['observations'][:5])

In [722]:
print(np.shape(data['desired_goals']))
# print(np.shape(data['sampled_desired_goals']))

In [723]:
# data['desired_goals'] = data['sampled_desired_goals']

In [724]:
print(data['desired_goals'][:3])
# print(data['sampled_desired_goals'][:3])

In [725]:
for i in range(5):
    print(data['observations'][i])
    print(data['actions'][i])

In [726]:
# first_action = np.zeros((1, data['actions'].shape[1]))
# all_actions = np.vstack((first_action, data['actions'][:-1]))
# obs_with_action = np.hstack((data['observations'], all_actions))
# 
# # Replace first action for every episode
# episode_ends = [index for index, value in enumerate(data['terminals']) if value]
# for idx in episode_ends[:-1]:
#     obs_with_action[idx+1][-2:] =  np.zeros((1, data['actions'].shape[1]))
# 
# data['observations'] = obs_with_action
# print(data['observations'].shape)

In [727]:
data['desired_goals'] = np.array([sub_array[:-2] for sub_array in data['desired_goals']])
print(np.shape(data['desired_goals']))
print(data['desired_goals'])

In [728]:
# print(data['actions'][:20,0])
# print(np.shape(data['actions']))
# print(np.shape(data['actions'][:,0].reshape(-1, 1)))
# print(data['actions'][:,0].reshape(-1, 1)[:20,0])

max = 0
min = 1
key = 'actions'
idx = 0
for i in range(len(data['terminals'])):
    if discretize_action_to_control_mode_E2E(data[key][i, 1])[1] == 4 or discretize_action_to_control_mode_E2E(data['actions'][i, 1])[1] == 5:
        # print(f"{i} Control mode: {discretize_action_to_control_mode_E2E(data['actions'][i, 1])[1]}, "
        #       # f"achieved: {data['observations'][i, 8:10]}, "
        #       # f"desired: {data['desired_goals'][i, :2]}, "
        #       f"indicator: {data['observations'][i,-3:]}, "
        #       f'action: {data["actions"][i]}'
        #       )
        if data[key][i][idx] > max:
            max = data[key][i][idx]
        if data[key][i][idx] < min:
            min = data[key][i][idx]
    # if int(data['terminals'][i]):
    #     print(f"Desired goal: {data['desired_goals'][i][:2]}")
    #     print(f"Episode End at {i}")
print(min, max)


# data['actions'] = [(data/(max-min))*2-1 if  for i, action in enumerate(data['actions'])]

In [729]:
''' Measure number of episodes '''
num = 0
for i, item in enumerate(data['terminals']):
    if item:
        num += 1
print(num)

In [730]:
''' Measure episode length '''
episode_len = []
last = 0
count = 0
for i, item in enumerate(data['terminals']):
    if item:
        episode_len.append(i - last)
        if i - last < 5:
            count += 1
            print(i, last)
        last = i
print("Mean: ", np.mean(episode_len))
print("STD: ", np.std(episode_len))
print("Max: ", np.max(episode_len))
print("Min: ", np.min(episode_len))
print(count)

In [731]:
''' Measure action size '''
print("Max: ", np.max(data['actions'][:, 0]))
print("Min: ", np.min(data['actions'][:, 0]))

In [732]:
# plt.figure(figsize=(10, 6))
# plt.plot((data['actions'][:,0]+1)/2, marker='o')
# plt.xlabel('Steps')
# plt.ylabel('Position')
# plt.title('Real-time Trajectory with Slow Acceleration and Slow Deceleration')
# plt.grid(True)
# plt.show()

In [733]:
''' Normalize state and desired goal '''
# def normalize(data, mean, std):
#     std = np.where(std == 0, 1, std)
#     return (data-mean)/std
# 
# goal_mean = np.mean(data['desired_goals'], axis=0)
# goal_std = np.std(data['desired_goals'], axis=0)
# desired_goal_norm = normalize(data['desired_goals'], goal_mean, goal_std)
# data['desired_goals'] = desired_goal_norm
# desired_goal_norm_param = {
#     'mean': goal_mean,
#     'std': goal_std,
# }
# 
# obs_mean = np.mean(data['observations'], axis=0)
# obs_std = np.std(data['observations'], axis=0)
# obs_norm = normalize(data['observations'], obs_mean, obs_std)
# data['observations'] = obs_norm
# obs_norm_param = {
#     'mean': obs_mean,
#     'std': obs_std,
# }

In [32]:
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/rotation_start_pose_prediction_dataset'
data = load_pickle(pickle_file_path)
for key in data.keys():
    print(np.shape(data[key]))

# Plot target pose region
def quaternion_to_yaw(w1, w2, w3, w4):
    # Quaternion to euler angle (yaw)
    siny_cosp = 2 * (w4 * w3 + w1 * w2)
    cosy_cosp = 1 - 2 * (w2 * w2 + w3 * w3)
    yaw = np.arctan2(siny_cosp, cosy_cosp)
    return yaw

plt.figure()
count = 0

for i in range(20):
    # Extract x, y, z and quaternion angle from the desired goals
    goal = data['actions'][i]
    x, y, z, w1, w2, w3, w4 = goal

    # Convert quaternion to yaw angle
    angle = quaternion_to_yaw(w1, w2, w3, w4)

    # Scatter plot for x and y
    plt.scatter(x, y, )

    # Calculate direction for quiver (assuming angle is in radians)
    u = np.cos(angle)
    v = np.sin(angle)

    # Quiver for the orientation
    plt.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=500, color='b')

    # if count > 10:
    #     break

print("Number of terminal states:", count)

plt.xlabel("y", fontsize='large', fontweight='bold')
plt.ylabel("x", fontsize='large', fontweight='bold')
plt.title("Target Position Distribution", fontweight='bold')
plt.grid(which='major')


In [735]:
''' Plot target pose region '''
plt.figure()
count = 0
for i, item in enumerate(data['terminals']):
    if item:
        plt.scatter(data['desired_goals'][i][0], data['desired_goals'][i][1], s=2)
        # print(data['desired_goals'][i])
        count += 1
    # if count > 50:
    #     break
print(count)
plt.xlabel("y", fontsize='large', fontweight='bold')
plt.ylabel("x", fontsize='large', fontweight='bold')
plt.title("Target Position Distribution", fontweight='bold')
plt.grid(which='major')
file_path = "/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/target_position_distribution.png"
# plt.savefig(file_path)
# plt.close()  # Good practice to close the plot to free resources
print("Plot saved successfully.")

# path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/result/VariableFriction.pth'
# data = torch.load(path, map_location=torch.device('cpu'))
# print(data.keys())
# print(data['state_normalizer_mean'], data['state_normalizer_std'])

In [ ]:
''' Print actions '''
# for i in range(10):
#     print(data['actions'][i])

In [ ]:
''' Plot target pose region '''
pickle_file_path = '/Users/qiyangyan/Desktop/Diffusion/Demonstration/VFF-bigSteps-2000demos-randomise'
data = load_pickle(pickle_file_path)
plt.figure()
count = 0
for i, item in enumerate(data['terminals']):
    if item:
        plt.scatter(data['desired_goals'][i][0], data['desired_goals'][i][1])
        print(data['desired_goals'][i])
        count += 1
    if count > 5:
        break
print(count)
plt.xlabel("y", fontsize='large', fontweight='bold')
plt.ylabel("x", fontsize='large', fontweight='bold')
plt.title("Target Position Distribution", fontweight='bold')
# plt.xlim([-2, 2])
# plt.ylim([-4, 2.5])
plt.grid(which='major')
file_path = "/Users/qiyangyan/Desktop/Training Files/Real4/Real4/demonstration/target_position_distribution.png"
# plt.savefig(file_path)
# plt.close()  # Good practice to close the plot to free resources
# print("Plot saved successfully.")

path = '/Users/qiyangyan/Desktop/Training Files/Real4/Real4/result/VariableFriction.pth'
data = torch.load(path, map_location=torch.device('cpu'))
print(data.keys())
print(data['state_normalizer_mean'], data['state_normalizer_std'])